In [27]:
from pymongo import MongoClient
import pandas as pd
import pprint
import json
import pandas as pd
import requests

In [28]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [29]:
not_dead = db.companies.find({"$and": [ {"offices":{"$not":{"$size":0}}}, {"deadpooled_year":None } ] })

In [30]:
df_entry = pd.DataFrame(not_dead)
df = df_entry.copy()
df.dropna(how='any')
df.shape
df.columns

Index(['_id', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
       'blog_url', 'category_code', 'competitions', 'created_at',
       'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds'],
      dtype='object')

In [31]:
def getLatLong(data):
    data = data['offices']
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "latitud": data[0]['latitude'],
        "longitud": data[0]['longitude'],
    }

latlong_office = df[["offices"]].apply(getLatLong, result_type="expand", axis=1)
# Y ahora mergeo con la tabla anterior y me quedo con estas columnas:
df = pd.concat([df,latlong_office], axis=1)[['_id',"name",
                                                   "latitud","longitud",'category_code',
                                                   'created_at','total_money_raised',
                                                   'description','number_of_employees',
                                                  'offices','funding_rounds']]

In [32]:
def RemoveWhiteSpaces(row):
    row = str(row)
    row = row.replace(" ","")
    return row
df['total_money_raised'] = df['total_money_raised'].apply(RemoveWhiteSpaces)

In [33]:
df['currency'] = df['total_money_raised'].str[0]
df['total_money_raised'] = df['total_money_raised'].str[1:]

In [34]:
def ChangeNonNumeric(row):
    row = str(row)
    try:
        if "B" in row:
            row = row.replace("B", "")
            row = float(row)*1000000000
        elif "M" in row:
            row = row.replace("M", "")
            row = float(row)*1000000
            
            
            
        elif "k" in row:
            row = row.replace("k", "")
            row = float(row)*1000
    except: row=None
    return row

In [35]:
df['total_money_raised'] = df['total_money_raised'].apply(ChangeNonNumeric)

In [128]:
def change_string(row):
    row = str(row)
    return row
def merge_currency_columns(df):
    df['currency'] = df['currency'].apply(change_string)
    df['total_money_raised'] = df['total_money_raised'].apply(change_string)
    df['currency_n'] = df['total_money_raised'] + df['currency']
    return df

In [97]:
# EXCHANGE RATE API
url = 'https://api.exchangerate-api.com/v4/latest/USD'
res = requests.get(url)
data = res.json()

In [125]:
def exchangeRate (row):
    row = str(row)
    try:
        if "$" in row:
            row = row.replace("$","")
            row = float(row)*data['rates']['USD']
        if "€" in row:
            row = row.replace("€","")
            row = float(row)*data['rates']['EUR']
        if "£" in row:
            row = row.replace("£","")
            row = float(row)*data['rates']['GBP']
        else:
            row = row.replace("$","")
            row = float(row)*data['rates']['USD']     
    except:
        row = row
    return row

In [126]:
def Apply_exchangeRate():
    df['currency_n'] = df['currency_n'].apply(exchangeRate)

In [130]:
df

,name,latitud,longitud,category_code,created_at,total_money_raised,description,number_of_employees,offices,funding_rounds,currency,currency_n
0,Digg,37.764726,-122.394523,news,Fri May 25 20:03:23 UTC 2007,45000000.0,user driven social content website,60.0,"[{'description': None, 'address1': '135 Missis...","[{'id': 1, 'round_code': 'b', 'source_url': 'h...",$,4.5e+07
1,Postini,37.506885,-122.247573,web,Fri Jun 08 12:19:51 UTC 2007,0,None,NaN,"[{'description': None, 'address1': '959 Skyway...",[],$,0
2,Flektor,34.025958,-118.379768,games_video,Thu May 31 21:11:51 UTC 2007,0,None,NaN,"[{'description': None, 'address1': '8536 Natio...",[],$,0
3,Geni,34.090368,-118.393064,web,Thu May 31 19:52:34 UTC 2007,16500000.0,Geneology social network site,18.0,"[{'description': 'Headquarters', 'address1': '...","[{'id': 6, 'round_code': 'a', 'source_url': ''...",$,1.65e+07
4,Fox Interactive Media,34.076179,-118.394170,web,Thu May 31 21:46:57 UTC 2007,0,,0.0,"[{'description': '', 'address1': '407 N Maple ...",[],$,0
5,Gizmoz,37.484130,-122.169472,web,Fri Jun 01 19:32:24 UTC 2007,18100000.0,Photo animation,NaN,"[{'description': None, 'address1': None, 'addr...","[{'id': 9, 'round_code': 'a', 'source_url': 'h...",$,1.81e+07
6,Helio,34.057498,-118.446596,mobile,Tue Jun 05 21:39:53 UTC 2007,0,Mobile Virtual Network Operator,NaN,"[{'description': None, 'address1': '10960 Wils...",[],$,0
7,StumbleUpon,37.775196,-122.419204,web,2007-06-01 09:28:13,18500000.0,Content discovery service,NaN,"[{'description': '', 'address1': '', 'address2...","[{'id': 8, 'round_code': 'seed', 'source_url':...",$,1.85e+07
8,Lala,37.451151,-122.154369,games_video,Tue Jun 05 12:20:29 UTC 2007,44200000.0,hub for online music discovery,NaN,"[{'description': 'Lala Headquarters', 'address...","[{'id': 16, 'round_code': 'b', 'source_url': '...",$,4.42e+07
9,Slacker,33.022176,-117.081406,music,Mon Jun 04 08:45:22 UTC 2007,73100000.0,Music Personalization Suite,NaN,"[{'description': '', 'address1': '16935 W. Ber...","[{'id': 12, 'round_code': 'b', 'source_url': '...",$,7.31e+07
